In [46]:
# Imports
import gym
import random
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [47]:
env_name = "FrozenLakeNoSlip-v0"
custom_map = ["SFFFFFFF", 
              "FFFFFFFF", 
              "FFFHFFFF",
              "FFFFFHFF", 
              "FFFHFFFF", 
              "FHHFFFHF", 
              "FHFFHFHF", 
              "FFFHFFFG"]

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'is_slippery':False},
        max_episode_steps=400,
        reward_threshold=0.78, # optimum = .8196
    )
except:
    pass

env = gym.make(env_name, desc = custom_map)
print("Observation space:", env.observation_space)
print("Action space:", env.action_space)
env.action_space

Observation space: Discrete(64)
Action space: Discrete(4)


C:\Users\DELL\AppData\Local\Programs\Python\Python39\lib\site-packages\gym\envs\registration.py:216: UserWarning: WARN: Overriding environment FrozenLakeNoSlip-v0
  logger.warn("Overriding environment {}".format(id))


Discrete(4)

In [48]:
import pygame, sys

SCREEN_WIDTH = 500
SCREEN_HEIGHT = 500

NUM_BLOCKS_WIDE = 8
NUM_BLOCKS_HIGH = 8
BLOCK_HEIGHT = round(SCREEN_HEIGHT/NUM_BLOCKS_HIGH)
BLOCK_WIDTH = round(SCREEN_WIDTH/NUM_BLOCKS_WIDE)

BLACK = (0, 0, 0)
START = (153, 184, 152) # color is white
GOAL = (0,201,87) # color is green
HOLE = (139,105,105) # color is brown
FROZEN = (24,116,205) # color is blue
PINK= (255,20,147) # color is pink

TITLE = "8x8 Forzen Lake"

def return_coordinates(state):
    count = 0
    matrix = []
    
    for _ in range(len(custom_map)):
        row = []
        for _ in range(len(custom_map[0])):
            row.append(count)
            count+=1
        matrix.append(row)
    
    for i in range(len(custom_map)):
        for j in range(len(custom_map[0])):
            if matrix[i][j] == state:
                pos = (i, j)
    
    return pos

In [49]:
def get_tile_color(tile):
    if tile == "S":
        color = START
    elif tile == "G":
        color = GOAL
    elif tile == "H":
        color = HOLE
    elif tile == "F":
        color = FROZEN
    else:
        color = PINK
    return color

In [50]:
def draw_map(surface, map_tiles):
    for j, tile in enumerate(map_tiles):
        for i, tile_contents in enumerate(tile):
            myrect = pygame.Rect(i * BLOCK_WIDTH, j * BLOCK_HEIGHT, BLOCK_WIDTH, BLOCK_HEIGHT)
            pygame.draw.rect(surface, get_tile_color(tile_contents), myrect)
            if get_tile_color(tile_contents) == PINK:
                helicopter = pygame.image.load('body.png')
                helicopter = pygame.transform.scale(helicopter, (BLOCK_WIDTH, BLOCK_WIDTH))
                surface.blit(helicopter, (i * BLOCK_WIDTH, j * BLOCK_HEIGHT))
            if get_tile_color(tile_contents) == HOLE:
                hole = pygame.image.load('hole1.jpg')
                hole = pygame.transform.scale(hole, (BLOCK_WIDTH, BLOCK_WIDTH))
                surface.blit(hole, (i * BLOCK_WIDTH, j * BLOCK_HEIGHT))
            if get_tile_color(tile_contents) == GOAL:
                hole = pygame.image.load('green_flag.png')
                hole = pygame.transform.scale(hole, (BLOCK_WIDTH, BLOCK_WIDTH))
                surface.blit(hole, (i * BLOCK_WIDTH, j * BLOCK_HEIGHT))
            if get_tile_color(tile_contents) == FROZEN:
                hole = pygame.image.load('frozen_lake.jpg')
                hole = pygame.transform.scale(hole, (BLOCK_WIDTH, BLOCK_WIDTH))
                surface.blit(hole, (i * BLOCK_WIDTH, j * BLOCK_HEIGHT))
            if get_tile_color(tile_contents) == START   :
                hole = pygame.image.load('start.jpg')
                hole = pygame.transform.scale(hole, (BLOCK_WIDTH, BLOCK_WIDTH))
                surface.blit(hole, (i * BLOCK_WIDTH, j * BLOCK_HEIGHT))

def draw_grid(surface):
    for i in range(NUM_BLOCKS_WIDE):
        new_height = round(i * BLOCK_HEIGHT)
        new_width = round(i * BLOCK_WIDTH)
        pygame.draw.line(surface, BLACK, (0, new_height), (SCREEN_WIDTH, new_height), 2)
        pygame.draw.line(surface, BLACK, (new_width, 0), (new_width, SCREEN_HEIGHT), 2)


In [51]:
def initialize_game():
    pygame.init()
    surface = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(TITLE)
    surface.fill(BLACK)
    return surface

def read_map(state_pos):
    world_map = custom_map
    world_map = []
    for row in custom_map:
        world_map.append(list(row))
        
    world_map[state_pos[0]][state_pos[1]] = "X"
    
    new_map = []
    for row in world_map:
        string = ""
        for ele in row:
            string += ele
        new_map.append(string)
    return new_map

In [52]:
import pickle
# with open("8x8_part_3frozen_lake_qtable.pkl", "wb") as f:
    # pickle.dump(agent.q_table, f)
with open("8x8_part_3frozen_lake_qtable.pkl", "rb") as f:
    loaded_table = pickle.load(f)
# loaded_table

In [53]:
def game_loop(surface):

    total_reward = 0
    rounds = 5
    num_eps = 400

    for i in range(1, rounds + 1):

        for j in range(1, num_eps + 1):
                    
            state = env.reset()
            done = False
            while not done:
                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit()
                action = np.argmax(loaded_table[state, :])
                new_state, reward, done, _ = env.step(action)
                state = new_state
                total_reward += reward

                print("j", j)
                print("s:", state, "a:", action)
                env.render()
                time.sleep(0.05)
                clear_output(wait=True)

                draw_map(surface, read_map(return_coordinates(state)))
                draw_grid(surface)
                pygame.display.update()

        total_reward = 0

In [54]:
world_map = custom_map
surface = initialize_game()
game_loop(surface)

SystemExit: 

C:\Users\DELL\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
